In [2]:
!pip install ultralytics

  Using cached opencv_python-4.10.0.84-cp37-abi3-win_amd64.whl.metadata (20 kB)
  Using cached torch-2.5.1-cp312-cp312-win_amd64.whl.metadata (28 kB)
  Using cached torchvision-0.20.1-cp312-cp312-win_amd64.whl.metadata (6.2 kB)
  Using cached ultralytics_thop-2.0.13-py3-none-any.whl.metadata (9.4 kB)
  Using cached sympy-1.13.1-py3-none-any.whl.metadata (12 kB)
   ---------------------------------------- 0.0/898.7 kB ? eta -:--:--
   ----------- ---------------------------- 262.1/898.7 kB ? eta -:--:--
   ---------------------------------------- 898.7/898.7 kB 2.1 MB/s eta 0:00:00
Using cached opencv_python-4.10.0.84-cp37-abi3-win_amd64.whl (38.8 MB)
Using cached torch-2.5.1-cp312-cp312-win_amd64.whl (203.0 MB)
Using cached sympy-1.13.1-py3-none-any.whl (6.2 MB)
Using cached torchvision-0.20.1-cp312-cp312-win_amd64.whl (1.6 MB)
Using cached ultralytics_thop-2.0.13-py3-none-any.whl (26 kB)
  Attempting uninstall: sympy
    Found existing installation: sympy 1.13.2
    Uninstalling sympy

In [1]:
import ultralytics
ultralytics.__version__

'8.3.49'

In [205]:
import cv2
import pandas as pd
from ultralytics import YOLO
from tracker import*
import os
import time

model=YOLO('yolov8s.pt')

In [206]:
class_list = ['person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed', 'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush']

In [222]:
count = 0
tracker = Tracker()
down = {}
up = {}
counter_down = []
counter_up = []
vehicle_types = {}
vehicle_classes = ['car', 'bus', 'truck', 'motorcycle']
speed_data = {}
vehicle_types = {}  # Menyimpan vehicle type yang paling sering untuk setiap id
vehicle_type_count = {}
vehicle_type_map = {}  # Menyimpan tipe kendaraan
counter_vehicle_types = {'car': 0, 'truck': 0, 'motorcycle': 0, 'bus': 0} 

In [223]:
red_line_y = 198
blue_line_y = 268
offset = 6

In [224]:
cap=cv2.VideoCapture('highway2.mp4')

In [225]:
# Create a folder to save frames
if not os.path.exists('detected_frames'):
    os.makedirs('detected_frames')

fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('output.avi', fourcc, 20.0, (1020, 500))

In [204]:
red_line_y = 198
blue_line_y = 268
offset = 6

while True:
    ret, frame = cap.read()
    if not ret:
        break
    
    frame = cv2.resize(frame, (1020, 500))

    # Prediksi menggunakan model
    results = model.predict(frame)
    a = results[0].boxes.data
    a = a.detach().cpu().numpy()
    px = pd.DataFrame(a).astype("float")
    
    list = []
    
    # Menyimpan tipe kendaraan untuk setiap kendaraan yang terdeteksi
    vehicle_type_map = {}

    for index, row in px.iterrows():
        x1 = int(row[0])
        y1 = int(row[1])
        x2 = int(row[2])
        y2 = int(row[3])
        d = int(row[5])
        c = class_list[d]
        
        if c in vehicle_classes:
            list.append([x1, y1, x2, y2])
            # Menampilkan tipe kendaraan di frame
            cv2.rectangle(frame, (x1, y1 - 20), (x1 + 100, y1), (0, 255, 0), -1)
            cv2.putText(frame, c, (x1, y1 - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 0), 1)

            # Menyimpan tipe kendaraan berdasarkan ID (dari posisi kendaraan yang terdeteksi)
            vehicle_type_map[(x1, y1, x2, y2)] = c  # Gunakan posisi kendaraan untuk pemetaan

    bbox_id = tracker.update(list)

    for bbox in bbox_id:
        x3, y3, x4, y4, id = bbox
        cx = int(x3 + x4) // 2
        cy = int(y3 + y4) // 2
    
        # Deteksi kendaraan yang bergerak ke bawah
        if red_line_y < (cy + offset) and red_line_y > (cy - offset):
            down[id] = cap.get(cv2.CAP_PROP_POS_MSEC) / 1000
        if id in down:
            if blue_line_y < (cy + offset) and blue_line_y > (cy - offset):
                elapsed_time = (cap.get(cv2.CAP_PROP_POS_MSEC) / 1000) - down[id]
                if counter_down.count(id) == 0:
                    counter_down.append(id)
                    distance = 10  # meters
                    a_speed_ms = distance / elapsed_time
                    a_speed_kh = a_speed_ms * 3.6
                    vehicle_type = vehicle_type_map.get((x3, y3, x4, y4), 'Unknown')  # Ambil tipe kendaraan dari pemetaan
                    speed_data[id] = (int(a_speed_kh), vehicle_type)
                    cv2.circle(frame, (cx, cy), 4, (0, 0, 255), -1)

                    # Update counter untuk tipe kendaraan
                    if vehicle_type in vehicle_classes:
                        counter_vehicle_types[vehicle_type] += 1
    
        # Deteksi kendaraan yang bergerak ke atas
        if blue_line_y < (cy + offset) and blue_line_y > (cy - offset):
            up[id] = cap.get(cv2.CAP_PROP_POS_MSEC) / 1000
        if id in up:
            if red_line_y < (cy + offset) and red_line_y > (cy - offset):
                elapsed1_time = (cap.get(cv2.CAP_PROP_POS_MSEC) / 1000) - up[id]
                if counter_up.count(id) == 0:
                    counter_up.append(id)
                    distance1 = 10  # meters
                    a_speed_ms1 = distance1 / elapsed1_time
                    a_speed_kh1 = a_speed_ms1 * 3.6
                    vehicle_type = vehicle_type_map.get((x3, y3, x4, y4), 'Unknown')  # Ambil tipe kendaraan dari pemetaan
                    speed_data[id] = (int(a_speed_kh1), vehicle_type)
                    cv2.circle(frame, (cx, cy), 4, (0, 0, 255), -1)

                    # Update counter untuk tipe kendaraan
                    if vehicle_type in vehicle_classes:
                        counter_vehicle_types[vehicle_type] += 1

        # Tampilkan kecepatan dan jenis kendaraan jika ada di speed_data
        if id in speed_data:
            speed, vehicle_type = speed_data[id]
            speed_text = f'{speed} Km/h ({vehicle_type})'
            cv2.rectangle(frame, (x3, y3 - 20), (x3 + 150, y3), (0, 255, 0), -1)
            cv2.putText(frame, speed_text, (x3, y3 - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 0), 1)

    # Gambar garis dan teks
    text_color = (0, 0, 0)
    yellow_color = (0, 255, 255)
    red_color = (0, 0, 255)
    blue_color = (255, 0, 0)

    cv2.rectangle(frame, (0, 0), (250, 120), yellow_color, -1)

    cv2.line(frame, (172, 198), (774, 198), red_color, 2)
    cv2.putText(frame, ('Red Line'), (172, 198), cv2.FONT_HERSHEY_SIMPLEX, 0.5, text_color, 1, cv2.LINE_AA)

    cv2.line(frame, (8, 268), (927, 268), blue_color, 2)
    cv2.putText(frame, ('Blue Line'), (8, 268), cv2.FONT_HERSHEY_SIMPLEX, 0.5, text_color, 1, cv2.LINE_AA)

    cv2.putText(frame, ('Going Down - ' + str(len(counter_down))), (10, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, text_color, 1, cv2.LINE_AA)
    cv2.putText(frame, ('Going Up - ' + str(len(counter_up))), (10, 40), cv2.FONT_HERSHEY_SIMPLEX, 0.5, text_color, 1, cv2.LINE_AA)

    # Tampilkan count untuk setiap jenis kendaraan
    y_offset = 60
    for vehicle_type, count in counter_vehicle_types.items():
        cv2.putText(frame, f'{vehicle_type.capitalize()} - {count}', (10, y_offset), cv2.FONT_HERSHEY_SIMPLEX, 0.5, text_color, 1, cv2.LINE_AA)
        y_offset += 20

    cv2.imshow("frames", frame)
    
    # Hentikan jika menekan tombol Escape
    if cv2.waitKey(1) & 0xFF == 27:
        break

cap.release()
cv2.destroyAllWindows()


0: 320x640 10 cars, 1 train, 1 truck, 60.5ms
Speed: 3.0ms preprocess, 60.5ms inference, 9.6ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 10 cars, 1 train, 1 truck, 69.6ms
Speed: 2.0ms preprocess, 69.6ms inference, 0.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 10 cars, 1 train, 1 truck, 67.5ms
Speed: 0.0ms preprocess, 67.5ms inference, 1.4ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 11 cars, 1 train, 1 truck, 79.7ms
Speed: 0.0ms preprocess, 79.7ms inference, 0.7ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 10 cars, 1 train, 1 truck, 67.2ms
Speed: 1.1ms preprocess, 67.2ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 10 cars, 1 train, 1 truck, 67.6ms
Speed: 0.0ms preprocess, 67.6ms inference, 0.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 10 cars, 1 train, 1 truck, 66.9ms
Speed: 0.0ms preprocess, 66.9ms inference, 1.2ms postprocess per image at shape (1, 3, 320, 640)

In [ ]:
red_line_y = 298
blue_line_y = 368
offset = 6

while True:
    ret, frame = cap.read()
    if not ret:
        break
    
    frame = cv2.resize(frame, (1020, 500))

    # Prediksi menggunakan model
    results = model.predict(frame)
    a = results[0].boxes.data
    a = a.detach().cpu().numpy()
    px = pd.DataFrame(a).astype("float")
    
    list = []
    
    # Menyimpan tipe kendaraan untuk setiap kendaraan yang terdeteksi
    vehicle_type_map = {}

    for index, row in px.iterrows():
        x1 = int(row[0])
        y1 = int(row[1])
        x2 = int(row[2])
        y2 = int(row[3])
        d = int(row[5])
        c = class_list[d]
        
        if c in vehicle_classes:
            list.append([x1, y1, x2, y2])
            # Menampilkan tipe kendaraan di frame
            cv2.rectangle(frame, (x1, y1 - 20), (x1 + 100, y1), (0, 255, 0), -1)
            cv2.putText(frame, c, (x1, y1 - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 0), 1)

            # Menyimpan tipe kendaraan berdasarkan ID (dari posisi kendaraan yang terdeteksi)
            vehicle_type_map[(x1, y1, x2, y2)] = c  # Gunakan posisi kendaraan untuk pemetaan

    bbox_id = tracker.update(list)

    for bbox in bbox_id:
        x3, y3, x4, y4, id = bbox
        cx = int(x3 + x4) // 2
        cy = int(y3 + y4) // 2
    
        # Deteksi kendaraan yang bergerak ke bawah
        if red_line_y < (cy + offset) and red_line_y > (cy - offset):
            down[id] = cap.get(cv2.CAP_PROP_POS_MSEC) / 1000
        if id in down:
            if blue_line_y < (cy + offset) and blue_line_y > (cy - offset):
                elapsed_time = (cap.get(cv2.CAP_PROP_POS_MSEC) / 1000) - down[id]
                if counter_down.count(id) == 0:
                    counter_down.append(id)
                    distance = 10  # meters
                    a_speed_ms = distance / elapsed_time
                    a_speed_kh = a_speed_ms * 3.6
                    vehicle_type = vehicle_type_map.get((x3, y3, x4, y4), 'Unknown')  # Ambil tipe kendaraan dari pemetaan
                    speed_data[id] = (int(a_speed_kh), vehicle_type)
                    cv2.circle(frame, (cx, cy), 4, (0, 0, 255), -1)

                    # Update counter untuk tipe kendaraan
                    if vehicle_type in vehicle_classes:
                        counter_vehicle_types[vehicle_type] += 1
    
        # Deteksi kendaraan yang bergerak ke atas
        if blue_line_y < (cy + offset) and blue_line_y > (cy - offset):
            up[id] = cap.get(cv2.CAP_PROP_POS_MSEC) / 1000
        if id in up:
            if red_line_y < (cy + offset) and red_line_y > (cy - offset):
                elapsed1_time = (cap.get(cv2.CAP_PROP_POS_MSEC) / 1000) - up[id]
                if counter_up.count(id) == 0:
                    counter_up.append(id)
                    distance1 = 10  # meters
                    a_speed_ms1 = distance1 / elapsed1_time
                    a_speed_kh1 = a_speed_ms1 * 3.6
                    vehicle_type = vehicle_type_map.get((x3, y3, x4, y4), 'Unknown')  # Ambil tipe kendaraan dari pemetaan
                    speed_data[id] = (int(a_speed_kh1), vehicle_type)
                    cv2.circle(frame, (cx, cy), 4, (0, 0, 255), -1)

                    # Update counter untuk tipe kendaraan
                    if vehicle_type in vehicle_classes:
                        counter_vehicle_types[vehicle_type] += 1

        # Tampilkan kecepatan dan jenis kendaraan jika ada di speed_data
        if id in speed_data:
            speed, vehicle_type = speed_data[id]
            speed_text = f'{speed} Km/h ({vehicle_type})'
            cv2.rectangle(frame, (x3, y3 - 20), (x3 + 150, y3), (0, 255, 0), -1)
            cv2.putText(frame, speed_text, (x3, y3 - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 0), 1)

    # Gambar garis dan teks
    text_color = (0, 0, 0)
    yellow_color = (0, 255, 255)
    red_color = (0, 0, 255)
    blue_color = (255, 0, 0)

    cv2.rectangle(frame, (0, 0), (250, 120), yellow_color, -1)

    cv2.line(frame, (172, 298), (774, 298), red_color, 2)
    cv2.putText(frame, ('Red Line'), (172, 298), cv2.FONT_HERSHEY_SIMPLEX, 0.5, text_color, 1, cv2.LINE_AA)

    cv2.line(frame, (8, 368), (927, 368), blue_color, 2)
    cv2.putText(frame, ('Blue Line'), (8, 368), cv2.FONT_HERSHEY_SIMPLEX, 0.5, text_color, 1, cv2.LINE_AA)

    cv2.putText(frame, ('Going Down - ' + str(len(counter_down))), (10, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, text_color, 1, cv2.LINE_AA)
    cv2.putText(frame, ('Going Up - ' + str(len(counter_up))), (10, 40), cv2.FONT_HERSHEY_SIMPLEX, 0.5, text_color, 1, cv2.LINE_AA)

    # Tampilkan count untuk setiap jenis kendaraan
    y_offset = 60
    for vehicle_type, count in counter_vehicle_types.items():
        cv2.putText(frame, f'{vehicle_type.capitalize()} - {count}', (10, y_offset), cv2.FONT_HERSHEY_SIMPLEX, 0.5, text_color, 1, cv2.LINE_AA)
        y_offset += 20

    cv2.imshow("frames", frame)
    
    # Hentikan jika menekan tombol Escape
    if cv2.waitKey(1) & 0xFF == 27:
        break

cap.release()
cv2.destroyAllWindows()


0: 320x640 10 cars, 1 train, 1 truck, 63.8ms
Speed: 2.9ms preprocess, 63.8ms inference, 0.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 10 cars, 1 train, 1 truck, 73.1ms
Speed: 2.0ms preprocess, 73.1ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 10 cars, 1 train, 1 truck, 68.9ms
Speed: 2.0ms preprocess, 68.9ms inference, 0.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 11 cars, 1 train, 1 truck, 75.1ms
Speed: 0.0ms preprocess, 75.1ms inference, 0.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 10 cars, 1 train, 1 truck, 70.2ms
Speed: 1.5ms preprocess, 70.2ms inference, 0.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 10 cars, 1 train, 1 truck, 69.1ms
Speed: 14.7ms preprocess, 69.1ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 10 cars, 1 train, 1 truck, 136.4ms
Speed: 1.0ms preprocess, 136.4ms inference, 0.0ms postprocess per image at shape (1, 3, 320, 6